In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
pd.options.display.max_rows = 999

In [2]:
accre_jobs = pd.read_csv('../data/accre_jobs_cleaned.csv')

# Jason --
    Question: 2   CMS Frequency of failure on particular nodes - user names: "cmslocal" and "cmspilot"

The CMS collaboration has an automated job submission system that runs jobs as "cmslocal" and "cmspilot". For these two users, jobs have internal system tests that will terminate their jobs early after approximately 30 minutes. Do any of their jobs that ended in under an hour also cluster on specific compute nodes, suggesting possbily unreliable systems? Check both “production” and “nogpfs” partitions. Look for commonly failing nodes and compare with other failed jobs.

In [3]:
# plt.style.use('dark_background')

In [4]:
cms_jobs_df = accre_jobs[(accre_jobs['user'] == "cmslocal") | (accre_jobs['user'] == "cmspilot")]

In [5]:
total_jobs = cms_jobs_df.shape[0]

In [6]:
cms_jobs_df['failure'] = np.where(cms_jobs_df['exitcode'] == "0:0", 0, 1)

<ipython-input-6-ee7b4d49ef10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cms_jobs_df['failure'] = np.where(cms_jobs_df['exitcode'] == "0:0", 0, 1)


In [7]:
cms_jobs_failed_df = cms_jobs_df[cms_jobs_df['failure'] == 1]

In [8]:
failures = cms_jobs_failed_df.shape[0]

In [9]:
failures / total_jobs * 100

0.027491310355378974

In [10]:
cms_jobs_df[cms_jobs_df['used_total_seconds'] < 3660].sort_values('used_total_seconds', ascending=False)

,Unnamed: 0,jobid,account,user,reqmem,usedmem,reqtime,usedtime,nodes,cpus,partition,exitcode,state,nodelist,req_total_seconds,used_total_seconds,mb_per_core,node_index,node_numbers,failure
402503,402503,17613549,cms,cmspilot,28000Mn,4745.66M,2-00:00:00,01:00:59,1,4,production,0:0,COMPLETED,cn1518,172800,3659,7000.000000,cn,['1518'],0
3599782,3599782,23738991,cms,cmspilot,28000Mn,5793.68M,2-00:00:00,01:00:59,1,4,production,0:0,COMPLETED,cn408,172800,3659,7000.000000,cn,['408'],0
3700442,3700442,23947055,cms,cmslocal,28000Mn,7649.01M,2-00:00:00,01:00:59,1,4,production,0:0,COMPLETED,cn392,172800,3659,7000.000000,cn,['392'],0
1894786,1894786,20352599,cms,cmslocal,5000Mn,1787.50M,2-00:00:00,01:00:59,1,1,production,0:0,COMPLETED,cn452,172800,3659,5000.000000,cn,['452'],0
274821,274821,17447254,cms,cmslocal,186648Mn,69683.26M,2-00:00:00,01:00:59,1,32,nogpfs,0:0,COMPLETED,cn1549,172800,3659,5832.750000,cn,['1549'],0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700725,700725,18040975,cms,cmslocal,38007Mn,0,2-00:00:00,00:00:00,1,8,nogpfs,0:0,COMPLETED,ng512,172800,0,4750.875000,ng,['512'],0
700726,700726,18040976,cms,cmslocal,42994Mn,0,2-00:00:00,00:00:00,1,12,nogpfs,0:0,COMPLETED,ng205,172800,0,3582.833333,ng,['205'],0
700732,700732,18040982,cms,cmspilot,21875Mn,0,2-00:00:00,00:00:00,1,8,nogpfs,0:0,COMPLETED,ng909,172800,0,2734.375000,ng,['909'],0
700738,700738,18040988,cms,cmspilot,38007Mn,0,2-00:00:00,00:00:00,1,8,nogpfs,0:0,COMPLETED,ng512,172800,0,4750.875000,ng,['512'],0


In [11]:
early_exits_df = cms_jobs_df[cms_jobs_df['used_total_seconds'] < 1920]

In [12]:
early_exits = early_exits_df.shape[0]

In [13]:
early_exits

481093

In [14]:
early_exits /total_jobs * 100

57.50381292521885

In [15]:
node_early_exists = early_exits_df['nodelist'].value_counts()

In [16]:
node_jobs = cms_jobs_df['nodelist'].value_counts()

In [17]:
node_jobs_df = pd.concat([node_early_exists, node_jobs], axis=1)

In [18]:
node_jobs_df.columns = ['early_exits', 'jobs']

In [19]:
node_jobs_df['failure_percent'] = node_jobs_df['early_exits'] / node_jobs_df['jobs'] * 100

In [20]:
node_jobs_df['failure_percent'].describe()

count    818.000000
mean      50.708806
std       17.515431
min       14.285714
25%       39.908123
50%       45.521436
75%       53.057066
max      100.000000
Name: failure_percent, dtype: float64

In [23]:
high_failure_nodes = node_jobs_df[node_jobs_df['failure_percent'] > 53.057066].index.to_list()

# Next Steps
- GO back and map high failure_nodes into the cms_jobs list and all the jobs and see if it dispropoiately fails.
- Map out failures by partition
- Map out failure clusters by order to job execution by order of job id

In [22]:
len(high_failure_nodes)

205